<a href="https://colab.research.google.com/github/shuchimishra/Tensorflow_projects/blob/main/Tensorflow_Code/Intro_to_ANN/MNIST_Grayscale_w_BasicConvolutions_&_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3: Improve MNIST with Convolutions

In the videos you looked at how you would improve Fashion MNIST using Convolutions. For this exercise see if you can improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model from the  assignment of the previous week.

You should stop training once the accuracy goes above this amount. It should happen in less than 10 epochs, so it's ok to hard code the number of epochs for training, but your training must end once it hits the above metric. If it doesn't, then you'll need to redesign your callback.

When 99.5% accuracy has been hit, you should print out the string "Reached 99.5% accuracy so cancelling training!"


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Load the data

Begin by loading the data. A couple of things to notice:

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)` but in this exercise you will be needing only the train set so you can ignore the second tuple.

In [ ]:
# Load the data
mnist = keras.datasets.mnist

# Get only training set
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#Let's view the shape of data
np.shape(train_images)

(60000, 28, 28)

In [ ]:
np.shape(test_images)

(10000, 28, 28)

## Pre-processing the data

One important step when dealing with image data is to preprocess the data. During the preprocess step you can apply transformations to the dataset that will be fed into your convolutional neural network.

Here you will apply two transformations to the data:
- Reshape the data so that it has an extra dimension. The reason for this
is that commonly you will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.


- Normalize the pixel values so that these are values between 0 and 1. You can achieve this by dividing every value in the array by the maximum.

Remember that these tensors are of type `numpy.ndarray` so you can use functions like [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) or [divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) to complete the `reshape_and_normalize` function below:

In [ ]:
# reshape_and_normalize function

def reshape_and_normalize(images):

    # Reshape the images to add an extra dimension
    images = np.reshape(images,(np.shape(images)[0],np.shape(images)[1],np.shape(images)[2],1))

    # Normalize pixel values
    images = images/255

    return images

Test your function with the next cell:

In [ ]:
# Apply your function
train_images = reshape_and_normalize(train_images)
test_images = reshape_and_normalize(test_images)

print(f"Maximum pixel value after normalization: {np.max(train_images)}\n")
print(f"Shape of training set after reshaping: {train_images.shape}\n")
print(f"Shape of one image after reshaping: {train_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining your callback

Now complete the callback that will ensure that training will stop after an accuracy of 99.5% is reached.

Define your callback in such a way that it checks for the metric `accuracy` (`acc` can normally be used as well but the grader expects this metric to be called `accuracy` so to avoid getting grading errors define it using the full word).

In [ ]:
# Defining the callback
class myCallback(keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        if (logs['accuracy'] > 0.995):
            print("Reached 99.5% accuracy so cancelling training!")
            self.model.stop_training=True


## Convolutional Model

Finally, complete the `convolutional_model` function below. This function should return your convolutional neural network.

**Your model should achieve an accuracy of 99.5% or more before 10 epochs to pass this assignment.**

**Hints:**
- You can try any architecture for the network but try to keep in mind you don't need a complex one. For instance, only one convolutional layer is needed.
- In case you need extra help you can check out an architecture that works pretty well at the end of this notebook.
- To avoid timeout issues with the autograder, please limit the number of units in your convolutional and dense layers. An exception will be raised if your model is too large.

In [ ]:
def convolutional_model():

    # Define the model
    model = tf.keras.models.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(32, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Save your untrained model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it '
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(train_images, train_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.1249 - accuracy: 0.9614
Epoch 2/10
1875/1875 [==============================] - 58s 31ms/step - loss: 0.0425 - accuracy: 0.9867
Epoch 3/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0277 - accuracy: 0.9913
Epoch 4/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0193 - accuracy: 0.9936
Epoch 5/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0164 - accuracy: 0.9944
Epoch 6/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0118 - accuracy: 0.9960


In [ ]:
#Print the model training output
print(history.__dir__())

['validation_data', 'model', '_chief_worker_only', '_supports_tf_logs', 'history', 'params', 'epoch', '__module__', '__doc__', '__init__', 'on_train_begin', 'on_epoch_end', '_keras_api_names', '_keras_api_names_v1', 'set_params', 'set_model', 'on_batch_begin', 'on_batch_end', 'on_epoch_begin', 'on_train_batch_begin', 'on_train_batch_end', 'on_test_batch_begin', 'on_test_batch_end', 'on_predict_batch_begin', 'on_predict_batch_end', 'on_train_end', 'on_test_begin', 'on_test_end', 'on_predict_begin', 'on_predict_end', '_implements_train_batch_hooks', '_implements_test_batch_hooks', '_implements_predict_batch_hooks', '__dict__', '__weakref__', '__new__', '__repr__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


In [ ]:
#Another effective way of training model training output
from pprint import pprint
pprint(vars(history))

{'_chief_worker_only': None,
 '_supports_tf_logs': False,
 'epoch': [0, 1, 2, 3, 4, 5],
 'history': {'accuracy': [0.9614333510398865,
                          0.9867333173751831,
                          0.9913166761398315,
                          0.9936166405677795,
                          0.9944000244140625,
                          0.9959999918937683],
             'loss': [0.12489030510187149,
                      0.04247314855456352,
                      0.027702094987034798,
                      0.01930655911564827,
                      0.0163822453469038,
                      0.011815344914793968]},
 'model': <keras.src.engine.sequential.Sequential object at 0x7a9b8990fbb0>,
 'params': {'epochs': 10, 'steps': 1875, 'verbose': 1},
 'validation_data': None}


If you see the message that you defined in your callback printed out after less than 10 epochs it means your callback worked as expected. You can also double check by running the following cell:

In [ ]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Your model was trained for 6 epochs


If your callback didn't stop training, one cause might be that you compiled your model using a metric other than `accuracy` (such as `acc`). Make sure you set the metric to `accuracy`. You can check by running the following cell:

In [ ]:
if not "accuracy" in history.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")

The metric was correctly defined.
